## Trying to get the annotations translated as well

In [1]:
annotation_map_TYPES = {
    "NOMBRE_SUJETO_ASISTENCIA": "NAME",
    "EDAD_SUJETO_ASISTENCIA": "AGE",
    "SEXO_SUJETO_ASISTENCIA": "SEX",
    "FAMILIARES_SUJETO_ASISTENCIA": "FAMILY",
    "NOMBRE_PERSONAL_SANITARIO": "PERSONNEL",
    "FECHAS": "DATE",
    "PROFESION": "PROFESSION",
    "HOSPITAL": "HOSPITAL",
    "ID_CENTRO_DE_SALUD": "ID",
    "INSTITUCION": "INSTITUTE",
    "CALLE": "STREET",
    "TERRITORIO": "TERRITORY",
    "PAIS": "COUNTRY",
    "NUMERO_TELEFONO": "TELEPHONE",
    "NUMERO_FAX": "FAX",
    "CORREO_ELECTRONICO": "EMAIL",
    "ID_SUJETO_ASISTENCIA": "ID",
    "ID_CONTACTO_ASISTENCIAL": "ID",
    "ID_ASEGURAMIENTO": "ID",
    "ID_TITULACION_PERSONAL_SANITARIO": "ID",
    "ID_EMPLEO_PERSONAL_SANITARIO": "ID",
    "IDENTIF_VEHÍCULOS_NRSERIE_PLACAS": "ID",
    "IDENTIF_DISPOSITIVOS_NRSERIE": "ID",
    "DIREC_PROT_INTERNET":"URL",
    "URL_WEB": "URL",
    "IDENTIF_BIOMETRICOS": "ID",
    "NUMERO_IDENTIF": "ID",
    "OTROS_SUJETO_ASISTENCIA": "OTHER",
}


# this works great
annotation_map = {
    "NOMBRE_SUJETO_ASISTENCIA": "#1",
    "EDAD_SUJETO_ASISTENCIA": "#2",
    "SEXO_SUJETO_ASISTENCIA": "#3",
    "FAMILIARES_SUJETO_ASISTENCIA": "#4",
    "NOMBRE_PERSONAL_SANITARIO": "#5",
    "FECHAS": "#6",
    "PROFESION": "#7",
    "HOSPITAL": "#8",
    "ID_CENTRO_DE_SALUD": "#9",
    "INSTITUCION": "#9",
    "CALLE": "#11",
    "TERRITORIO": "#12",
    "PAIS": "#13",
    "NUMERO_TELEFONO": "#14",
    "NUMERO_FAX": "#15",
    "CORREO_ELECTRONICO": "#16",
    "ID_SUJETO_ASISTENCIA": "#17",
    "ID_CONTACTO_ASISTENCIAL": "#18",
    "ID_ASEGURAMIENTO": "#19",
    "ID_TITULACION_PERSONAL_SANITARIO": "#20",
    "ID_EMPLEO_PERSONAL_SANITARIO": "#21",
    "IDENTIF_VEHÍCULOS_NRSERIE_PLACAS": "#22",
    "IDENTIF_DISPOSITIVOS_NRSERIE": "#23",
    "DIREC_PROT_INTERNET":"#24",
    "URL_WEB": "#25",
    "IDENTIF_BIOMETRICOS": "#26",
    "NUMERO_IDENTIF": "#27",
    "OTROS_SUJETO_ASISTENCIA": "#28",
}

def apply_annotations(text_file_path, annotations_file_path, output_file_path):
    # Read the text file
    with open(text_file_path, 'r') as text_file:
        text = text_file.read()
    
    # Read the annotations file
    annotations = []
    with open(annotations_file_path, 'r') as ann_file:
        for line in ann_file:
            #print(line.split('\t'))
            parts = line.split('\t')
            if len(parts) == 3:
                index = parts[0]
                (annotation_type, index_start, index_end) = (i for i in parts[1].split(" "))
                annotation_type = annotation_map[annotation_type]
                annotated_text = parts[2].replace("\n","")
                annotations.append((index, annotation_type, int(index_start), int(index_end), annotated_text))
    
    # Sort annotations by the start index (to avoid overlap issues when applying them)
    annotations.sort(key=lambda x: x[2])  # Sort by index_start
    #print(annotations)

    # Apply the annotations to the text
    offset = 0 # This accounts for the changes in the text length due to annotations
    for index, annotation_type, index_start, index_end, annotated_text in annotations:
        # Adjust the indices due to the changes in text length as annotations are applied
        start_adjusted = index_start + offset
        end_adjusted = index_end + offset
        
        # Replace the segment with the annotation
        text = text[:start_adjusted] + f"[{annotation_type} {annotated_text}]" + text[end_adjusted:]
        
        # Update the offset (length of the annotation replaces the old text length)
        offset += len(f"[{annotation_type}] {annotated_text}") - (end_adjusted - start_adjusted)
    
    # Save the annotated text to a new file
    #with open(output_file_path, 'w') as output_file:
        #output_file.write(text)
    return text

# Example usage
text_file_path = '/scratch/project_2009498/SPACCC_MEDDOCAN/corpus/dev/brat/S0004-06142006000500012-1.txt'
annotations_file_path = '/scratch/project_2009498/SPACCC_MEDDOCAN/corpus/dev/brat/S0004-06142006000500012-1.ann'
output_file_path = 'output_text_with_annotations.txt'
redacted = apply_annotations(text_file_path, annotations_file_path, output_file_path)
print(redacted)


Datos del paciente.
Nombre:  [#1 Pedro].
Apellidos: [#1 De Miguel Rivera].
NHC: [#17 2569870].
Domicilio: [#11 Calle Carmen Romero, 23, 1D].
Localidad/ Provincia: [#12 Madrid].
CP: [#12 28035].
Datos asistenciales.
Fecha de nacimiento: [#6 10/10/1963].
País: [#13 España].
Edad: [#2 53 años] Sexo: [#3 H].
Fecha de Ingreso: [#6 17/06/2016].
Médico: [#5 Estefanía Romero Selas]  NºCol: [#20 28 28 20943].
Informe clínico del paciente: [#3 varón] de [#2 53 años] sin antecedentes de interés que ingresa procedente de urgencias con un cuadro de tromboembolismo pulmonar.
Ante la sospecha de neoplasia oculta y la presencia de hematuria no evidenciada previamente se procede a la realización de Ecografía abdominal donde se evidencia una masa renal derecha y se completa el estudio con TAC y RM..
Ambos estudios confirman la presencia de una tumoración heterogénea que infiltra los dos tercios inferiores del riñón derecho de aproximadamente 10x10 cms. con afectación del seno e hilio renal objetivándose

## The other way round!!!!


In [64]:
translated_file="../translated_with_annotationsFI.txt"
reference_file="../translated/fi/dev/S0004-06142006000500012-1.txt"
import re

In [65]:
def reverse_annotations(input_file, reference_file, output_file):
    
    with open(input_file,'r') as file:
        text = file.read()

    with open(reference_file, 'r') as file:
        ref = file.read()
    #i = re.finditer(r'\[\#[0-9]{1,2}([ ]\w+)+\]', text)
    i = re.finditer(r'\[\#[0-9]{1,2}[ ](\w+,?.)+\]', text)
    #print([text[m.start(0):m.end(0)] for m in i])
    ind = [[m.start(0),m.end(0)] for m in i]
    #print(ind)
    current_index=0
    parsed_text=""
    annotations=[]
    #print(i)
    for start, end in ind:
        parsed_text = parsed_text + text[current_index:start]
        #print(parsed_text)
        annot = text[start:end].split(" ")
        pii_class = annot[0].replace("[","")
        annot_text = " ".join(annot[1:]).replace("]","")
        #print(pii_class)
        print(annot_text)
        print("")
        parsed_text += annot_text
        current_index = end
        annotations.append([pii_class, len(parsed_text)-len(annot_text), len(parsed_text)])
        #assert parsed_text == ref[0:current_index], f'\n{parsed_text} \n{ref[0:current_index]}'
    print(parsed_text)
    print(ref)
    print(annotations)
    assert parsed_text==ref

reverse_annotations(translated_file, reference_file, "testi")
        

[[26, 36], [48, 69], [76, 89], [99, 132], [154, 166], [172, 183], [211, 226], [233, 246], [253, 267], [279, 288], [308, 323], [334, 361], [370, 387], [420, 429], [438, 452], [3040, 3067], [3081, 3108]]
Pedro

De Miguel Rivera

2569870

Calle Carmen Romero, 23, 1D

Madrid

28035

10/10/1963

Espanja

53 vuotta

mies

17/06/2016

Estefanía Romero Selas

28 28 20943

mies

53 vuotta

Estefanía Romero Selas

eromeroselas@yahoo.es

Potilaan tiedot.
Etunimi: Pedro.
Sukunimi: De Miguel Rivera.
NHC: 2569870.
Osoite: Calle Carmen Romero, 23, 1D.
Kaupunki/ maakunta: Madrid.
CP: 28035.
Hoitotiedot.
Syntymäaika: 10/10/1963.
Maa: Espanja.
Ikä: 53 vuotta Sukupuoli: mies.
Vastaanottopäivä: 17/06/2016.
Lääkäri: Estefanía Romero Selas Col Nº: 28 28 20943.
Kliininen raportti potilaasta: mies iältään 53 vuotta, jolla ei ole aiempaa sairaushistoriaa ja joka otettiin vastaan päivystysosastolta keuhkotromboembolian vuoksi.
Koska epäiltiin piilevää kasvainta ja aiemmin havaitsematonta hematuriaa, tehtiin vat

AssertionError: 